In [ ]:
!pip install scikit-learn
!pip install wordcloud
!pip install nltk
!pip install seaborn
!pip install unidecode

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import nltk
import seaborn as sns
import unidecode
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import wordcloud
from nltk import tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams


In [ ]:
# nltk.download("all")
# nltk.download("ptb") 

In [ ]:
# os.getcwd()

In [ ]:
reviews = pd.read_csv("data/imdb-reviews-pt-br.csv")
reviews.head(5)

In [ ]:
# # from sklearn.model_selection import train_test_split
# train, test, class_train, class_test = train_test_split(reviews.text_pt, reviews.sentiment, random_state = 42)

# # from sklearn.linear_model import LogisticRegression
# RegressionLogistic = LogisticRegression()
# RegressionLogistic.fit(train, class_train)
# acuracy = RegressionLogistic.score(test, class_test)
# print(acuracy)

In [ ]:
reviews.sentiment.value_counts()

In [ ]:
classification = reviews['sentiment'].replace(['neg','pos'],[0,1])
reviews['classification'] = classification
reviews.head(5)

In [ ]:
text = ['Assisti um filme maravilhoso','Assisti um filme muito bom','Filme péssimo']
vetorize = CountVectorizer(lowercase=False)
bag_of_words = vetorize.fit_transform(text)
vetorize.get_feature_names_out()

In [ ]:
matrix_sparce = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns=vetorize.get_feature_names_out())
matrix_sparce

In [ ]:
# vetorize = CountVectorizer(lowercase=False)
#vetorize = CountVectorizer()
# vetorize = CountVectorizer(max_features=100)
# bag_of_words = vetorize.fit_transform(reviews.text_pt)
# bag_of_words.shape

In [ ]:
vetorize = CountVectorizer(max_features=100)
bag_of_words = vetorize.fit_transform(reviews.text_pt)
bag_of_words.shape
train, test, class_train, class_test = train_test_split(bag_of_words, reviews.classification, random_state = 42)
RegressionLogistic = LogisticRegression(max_iter=100)
RegressionLogistic.fit(train, class_train)
acuracy = RegressionLogistic.score(test, class_test)
print(acuracy)

In [ ]:
vetorize = CountVectorizer(max_features=300)
bag_of_words = vetorize.fit_transform(reviews.text_pt)
bag_of_words.shape
train, test, class_train, class_test = train_test_split(bag_of_words, reviews.classification, random_state = 42)
RegressionLogistic = LogisticRegression(max_iter=300)
RegressionLogistic.fit(train, class_train)
acuracy = RegressionLogistic.score(test, class_test)
print(acuracy)

In [ ]:
vetorize = CountVectorizer(max_features=500)
bag_of_words = vetorize.fit_transform(reviews.text_pt)
bag_of_words.shape
train, test, class_train, class_test = train_test_split(bag_of_words, reviews.classification, random_state = 42)
RegressionLogistic = LogisticRegression(max_iter=500)
RegressionLogistic.fit(train, class_train)
acuracy = RegressionLogistic.score(test, class_test)
print(acuracy)

In [ ]:
vetorize = CountVectorizer(max_features=1000)
bag_of_words = vetorize.fit_transform(reviews.text_pt)
bag_of_words.shape
train, test, class_train, class_test = train_test_split(bag_of_words, reviews.classification, random_state = 42)
# RegressionLogistic = LogisticRegression(max_iter=500)
RegressionLogistic = LogisticRegression(max_iter=1000,solver='saga')
RegressionLogistic.fit(train, class_train)
acuracy = RegressionLogistic.score(test, class_test)
print(acuracy)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all reviews into one string
all_words = ' '.join([text for text in reviews.text_pt])

# Generate the word cloud
word_cloud = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(all_words)

# Display the word cloud
plt.figure(figsize=(10,7))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
# type(all_words)
len(all_words)

In [ ]:
def cloudWordsNegative(text, column_text):
    text_negative = text.query("sentiment == 'neg'")
    # Combine all reviews into one string
    all_words = ' '.join([text for text in text_negative[column_text]])

    # Generate the word cloud
    word_cloud = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(all_words)

    # Display the word cloud
    plt.figure(figsize=(10,7))
    plt.imshow(word_cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

def cloudWordsPositive(text, column_text):
    text_positive = text.query("sentiment == 'pos'")
    # Combine all reviews into one string
    all_words = ' '.join([text for text in text_positive[column_text]])

    # Generate the word cloud
    word_cloud = WordCloud(width=800, height=500, max_font_size=110, collocations=False).generate(all_words)

    # Display the word cloud
    plt.figure(figsize=(10,7))
    plt.imshow(word_cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

In [ ]:
cloudWordsNegative(reviews,"text_pt")

In [ ]:
cloudWordsPositive(reviews,"text_pt")

In [ ]:
token_space = tokenize.WhitespaceTokenizer()
token_reviews = token_space.tokenize(all_words)
frequency = nltk.FreqDist(token_reviews)
df_frequency = pd.DataFrame({
                                'Word': list(frequency.keys()),
                                'Frequency': list(frequency.values())
                            })
df_frequency.nlargest(columns= "Frequency", n = 5)


In [ ]:
# def graphBar(df_frequency):
def graphBar(df, df_column, quantity):
    all_words = ' '.join([df for df in df[df_column]])
    token_space = tokenize.WhitespaceTokenizer()
    token_reviews = token_space.tokenize(all_words)
    frequency = nltk.FreqDist(token_reviews)
    df_frequency = pd.DataFrame({
                                    'Word': list(frequency.keys()),
                                    'Frequency': list(frequency.values())
                                })
    df2 = df_frequency.nlargest(columns= "Frequency", n = quantity)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df2, x= "Word", y = "Frequency", color = "gray")
    ax.set(ylabel = "Count")
    plt.show()

In [ ]:
graphBar(reviews, "text_pt", 20)
# graphBar(df_frequency.nlargest(columns= "Frequency", n = 20))

In [ ]:
irrelevant_word = nltk.corpus.stopwords.words("portuguese")
print(irrelevant_word)

In [ ]:
# reviews.drop(columns=['tratamento_1'], inplace=True)

In [ ]:
# def removeStopWords(df):
#     irrelevant_word = nltk.corpus.stopwords.words("portuguese")
#     line_string = list()
#     for review in df.text_pt:
#         new_line = list()
#         token_space = tokenize.WhitespaceTokenizer()
#         new_line = token_space.tokenize(review)
#         for word in new_line:
#             if word not in irrelevant_word:
#                 new_line.append(word)
#         line_string.append(' '.join(new_line))
#     df['tratamento_1'] = line_string

# removeStopWords(reviews)

In [ ]:
# Second Version
def removeStopWords(df):
    # Load stopwords
    irrelevant_words = nltk.corpus.stopwords.words("portuguese")
    tokenizer = WhitespaceTokenizer()
    # Create a list to store the processed reviews
    line_string = []
    # Iterate through each review
    for review in df['text_pt']:
        # Tokenize the review into words
        words_line = tokenizer.tokenize(review)
        # Remove stopwords
        new_line = [word for word in words_line if word not in irrelevant_words]
        # Join the words back into a string and add to the list
        line_string.append(' '.join(new_line))
    # Add the processed reviews as a new column
    df['treatment'] = line_string

removeStopWords(reviews)


In [ ]:
reviews.head(5)

In [ ]:
def classification(df, df_column, df_column_classif, iteration):
    vetorize = CountVectorizer(max_features=iteration)
    bag_of_words = vetorize.fit_transform(df[df_column])
    bag_of_words.shape
    train, test, class_train, class_test = train_test_split(bag_of_words, df[df_column_classif], random_state = 42)
    # RegressionLogistic = LogisticRegression(max_iter=500)
    RegressionLogistic = LogisticRegression(max_iter=iteration,solver='saga')
    RegressionLogistic.fit(train, class_train)
    acuracy = RegressionLogistic.score(test, class_test)
    print(acuracy)

In [ ]:
classification(reviews, "treatment","classification",500)

In [ ]:
graphBar(reviews,"treatment",20)

In [ ]:

# frase = "Os cursos da Alura são ótimos, além de ótimos, têm alunos ótimos!"
# token_espaco = WhitespaceTokenizer()
# token_pontuacao = WordPunctTokenizer()

# token_1 = token_espaco.tokenize(frase)
# token_2 = token_pontuacao.tokenize(frase)
# print(token_1)
# print(token_2)

In [ ]:
def treatmentsToClean(df, df_column):
    token_punctuation = WordPunctTokenizer()
    stemmer = nltk.RSLPStemmer()

    # Aggregate a DataFrame List of Punctuation to remove
    characters_and_words_to_remove = []
    for punct in punctuation:
        characters_and_words_to_remove.append(punct)

    # Aggregate a DataFrame List of Punctuation + Irrelevant Words to remove
    characters_and_words_to_remove = characters_and_words_to_remove + irrelevant_word

    # Aggregate a DataFrame List of Punctuation + Irrelevant Words to remove + Words without Accents
    characters_and_words_to_remove = [unidecode.unidecode(review) for review in characters_and_words_to_remove]

    new_review_ok = []
    for review in df[df_column]:
        new_review = []
        review = review.lower()
        words_review = token_punctuation.tokenize(review)
        for word in words_review:
            if word not in characters_and_words_to_remove:
                # Alter eath word for their prefix with the "nltk.RSLPStemmer.stem(word)"
                new_review.append(stemmer.stem(word))
        new_review_ok.append(' '.join(new_review))

    df[df_column] = new_review_ok

In [ ]:
treatmentsToClean(reviews,'treatment')

In [ ]:
classification(reviews, "text_pt","classification",500)

In [ ]:
classification(reviews, "treatment","classification",500)

In [ ]:
# stemmer = nltk.RSLPStemmer()
# a = stemmer.stem("corredor")
# b = stemmer.stem("corre")
# c = stemmer.stem("correria")
# d = stemmer.stem("correlacionado")
# e = stemmer.stem("corrimão")
# print(a,b,c,d,e)

In [ ]:
sentences = ['Assisti um ótimo filme','Assisti um filme péssimo']
tfidf = TfidfVectorizer(max_features=500)
specs = tfidf.fit_transform(sentences)
pd.DataFrame(
        specs.todense(),
        columns=tfidf.get_feature_names_out()
)

In [ ]:
tfidf_original = tfidf.fit_transform(reviews['text_pt'])
train, test, class_train, class_test = train_test_split(tfidf_original, reviews['classification'], random_state=42)

RegressionLogistic.fit(train, class_train)
acuracy_tfidf_original = RegressionLogistic.score(test, class_test)
print(acuracy_tfidf_original)


In [ ]:
tfidf_treated = tfidf.fit_transform(reviews['treatment'])
train, test, class_train, class_test = train_test_split(tfidf_treated, reviews['classification'], random_state=42)

RegressionLogistic.fit(train, class_train)
acuracy_tfidf_treated = RegressionLogistic.score(test, class_test)
print(acuracy_tfidf_treated)


In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
vector_tfidf = tfidf.fit_transform(reviews['treatment'])
train, test, class_train, class_test = train_test_split(vector_tfidf, reviews['classification'],random_state=42)
RegressionLogistic.fit(train, class_train)
acuracy_tfidf_treated_ngrams = RegressionLogistic.score(test, class_test)
print(acuracy_tfidf_treated_ngrams)

In [ ]:
tfidf = TfidfVectorizer()
vector_tfidf = tfidf.fit_transform(reviews['treatment'])
train, test, class_train, class_test = train_test_split(vector_tfidf, reviews['classification'],random_state=42)
RegressionLogistic.fit(train, class_train)
acuracy_tfidf_treated_no_ngrams = RegressionLogistic.score(test, class_test)
print(acuracy_tfidf_treated_no_ngrams)

In [ ]:
weight_pos = pd.DataFrame(
    RegressionLogistic.coef_[0].T,
    index= tfidf.get_feature_names_out()
)
weight_pos.nlargest(10,0)

In [ ]:
weight_neg = pd.DataFrame(
    RegressionLogistic.coef_[0].T,
    index= tfidf.get_feature_names_out()
)
weight_neg.nsmallest(10,0)